# AutoML ONNX: Train "the best" classifier model for the Iris dataset.

この Notebook では Open Neural Network Exchange (ONNX) を使用して、Azure Machine Learning の自動機械学習 (AutoML) から生成されたモデルで予測を行う方法について説明します。

必要条件 - このチュートリアルの恩恵を受けるためには、以下のものが必要です：

- 機械学習の基本的な理解
- アクティブなサブスクリプションを持つAzureアカウント。[無料でアカウントを作成](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- Azure ML ワークスペース。ワークスペースの作成については、[このノートブック](https://github.com/Azure/azureml-examples/blob/013bff90530ac920909132ef0362049d63151a40/sdk/python/resources/workspace/workspace.ipynb)を参照してください。
- コンピュートクラスタ。コンピュートクラスタを作成するには、[このノートブック](https://github.com/Azure/azureml-examples/blob/013bff90530ac920909132ef0362049d63151a40/sdk/python/resources/compute/compute.ipynb)を参照してください。
- Python 環境
- Azure Machine Learning Python SDK v2 をインストール - インストール手順 - [Getting started](https://github.com/Azure/azureml-examples/blob/013bff90530ac920909132ef0362049d63151a40/sdk/python/README.md) セクションを確認してください。<br>
Azure Machine Learning Notebooks には既にインストールされています。右上から `Python 3.10 - SDK v2` カーネルを選択します。


## 1. Connect to Azure Machine Learning Workspace
### Configure workspace details and get a handle to the workspace
ワークスペースは、Azure Machine Learning の最上位リソースであり、Azure Machine Learning を使用するときに作成するすべての成果物で作業するための一元的な場所を提供します。このセクションでは、ジョブを実行するワークスペースに接続します。

ワークスペースに接続するには、識別子パラメーター（サブスクリプション、リソースグループ、ワークスペース名）が必要です。`azure.ai.ml` の `MLClient` でこれらの詳細を使用して、必要な Azure Machine Learning ワークスペースへのハンドルを取得します。このチュートリアルでは、デフォルトの [Azure 認証](https://learn.microsoft.com/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python)を使用します。認証情報を設定してワークスペースに接続する方法の詳細については、[設定ノートブック](https://github.com/Azure/azureml-examples/blob/013bff90530ac920909132ef0362049d63151a40/sdk/python/jobs/configuration.ipynb)を確認してください。

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

credential = DefaultAzureCredential()
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your Azure Machine Learning workspace
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP>"
    workspace = "<AML_WORKSPACE_NAME>"

    ml_client = MLClient(credential, subscription_id, resource_group, workspace)

print(ml_client.workspace_name, ml_client.connections._subscription_id, sep = '\n')

### Show Azure ML Workspace information

In [ ]:
workspace = ml_client.workspaces.get(name=ml_client.workspace_name)

subscription_id = ml_client.connections._subscription_id
resource_group = workspace.resource_group
workspace_name = ml_client.workspace_name

output = {}
output["Workspace"] = workspace_name
output["Subscription ID"] = subscription_id
output["Resource Group"] = resource_group
output["Location"] = workspace.location
output

## 2. MLTable with input Training Data

### 2.1. Register training data
省略。[sklearn.datasets.load_iris](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html) を Machine Learning Studio UI から登録済み。

### 2.2. Load training data

In [ ]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes

registered_data_asset = ml_client.data.get(name='Iris-cleaned', version="1")
my_training_data_input = Input(
    type=AssetTypes.MLTABLE, 
    path=registered_data_asset.id
)

my_training_data_input

## 3. Compute target setup

### Use existing compute target or create new (Basic)

Azure Machine Learning Computeは、適切な VM Family のシングルノードからマルチノードコンピュートまでを簡単に作成できるマネージドコンピュートインフラストラクチャです。**ワークスペース・リージョン内**に作成され、ワークスペース内の他のユーザーが使用できるリソースです。ジョブが投入されると、デフォルトで max_nodes までオートスケールされ、ユーザが指定した依存関係をパッケージングしたコンテナ化環境で実行されます。

マネージドコンピュートなので、ジョブのスケジューリングとクラスタの管理は Azure Machine Learning サービスが内部で行う。

コンピュートクラスタは `AmlCompute` クラスを使用して作成できる。このクラスの主要なパラメータは以下の通りである：

* `size` - クラスタに使用する VM のサイズ。詳細については、[Supported VM series and sizes](https://docs.microsoft.com/azure/machine-learning/concept-compute-target#supported-vm-series-and-sizes) を参照してください。
* `max_instances` - クラスタで使用するノードの最大数。デフォルトは 1。

In [ ]:
from azure.ai.ml.entities import AmlCompute

# Choose a name for your CPU cluster
cluster_name = "ClusterDS3"

# Verify that cluster does not exist already
try:
    cluster = ml_client.compute.get(cluster_name)
    print('Found existing cluster, use it.')
except Exception:
    compute = AmlCompute(name=cluster_name, size='STANDARD_DS3_V2',
                         max_instances=4)
    cluster = ml_client.compute.begin_create_or_update(compute)


## 4. Configure and run the AutoML classification job

### classification() parameters:

`classification()` ファクトリ関数を使うと、最も一般的なシナリオの分類タスク用に AutoML を設定することができます。

- `target_column_name` - 予測の対象となるカラムの名前。常に指定する必要がある。このパラメータは 'training_data'、'validation_data'、'test_data' に適用できる。
- `primary_metric` - 分類モデル選択のために AutoML が最適化する指標。
- `training_data` - 学習に使用するデータ。トレーニング特徴列とターゲット列の両方を含む必要がある。オプションとして、このデータを分割して検証データセットやテストデータセットとすることもできる。
ワークスペースに登録されている MLTable を '<mltable_name>:<version>' というフォーマットで使用するか、ローカルのファイルやフォルダを MLTable として使用することができます。例： Input(mltable='my_mltable:1') OR Input(mltable=MLTable(local_path="./data"))
パラメータ 'training_data' は必ず指定すること。
- `compute` - AutoMLジョブが実行されるコンピュートターゲット。この例では、ワークスペースに存在する 'cpu-cluster' というコンピュートを使っています。ワークスペース内の他のコンピュートと置き換えることもできます。
- `name` - ジョブ/ランの名前です。これはオプションのプロパティです。指定しない場合は、ランダムな名前が生成される。
- `experiment_name` - 実験の名前。Experiment は、Azure ML Workspace 内にある、同じ論理的な機械学習実験に関連する複数の実行を含むフォルダのようなものです。


### set_limits() function parameters:
これはタイムアウトなどの制限パラメータを設定するためのオプションの設定方法です。    
    
- `timeout_minutes` - AutoML ジョブ全体が終了するまでの最大時間（分）。このタイムアウトには、セットアップ、特徴量化、トレーニングの実行は含まれるが、処理の最後に行われるアンサンブルとモデルの説明可能性の実行は含まれない。指定しない場合、デフォルトのジョブの合計タイムアウトは6日（8,640分）です。1時間（60分）以下のタイムアウトを指定するには、データセットのサイズが10,000,000（行×列）以下であることを確認してください。

- `trial_timeout_minutes` - 各トライアル（子ジョブ）が終了するまでの最大時間（分）。指定しない場合は、1ヶ月または 43200 分の値が使用される。
    
- `max_trials` - 1つの AutoML ジョブで、アルゴリズムとハイパーパラメータの組み合わせを変えて試行/実行する回数の最大値。指定しない場合、デフォルトは 1000 試行である。enable_early_termination' を使用する場合は、試行回数を少なくすることができる。
    
- `max_concurrent_trials` - 並列実行されるトライアル（子ジョブ）の最大数を表す。この数とクラスタのノード数を一致させるのが良い方法である。
    
- `enable_early_termination` - スコアが短期的に向上しない場合に、早期終了を有効にするかどうか。


In [ ]:
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import automl, Input

# note that this is a code snippet -- you might have to modify the variable values to run it successfully
# configure the classification job
classification_job = automl.classification(
    compute='ClusterDS3',
    experiment_name='IrisClassifySDK',
    training_data=my_training_data_input,
    target_column_name="Species",
    primary_metric="accuracy",
    n_cross_validations=5,
    enable_model_explainability=True,
    tags={"my_custom_tag": "SDK v2"}
)

# Limits are all optional
classification_job.set_limits(
    timeout_minutes=600, 
    trial_timeout_minutes=20, 
    max_trials=5,
    enable_early_termination=True,
)

# Training properties are optional
classification_job.set_training(
    blocked_training_algorithms=["logistic_regression"], 
    enable_onnx_compatible_models=True #ONNX 互換モデルの生成を有効にする
)



## 5. Run the Command
先ほど作成した `MLClient` を使って、ワークスペースでこのコマンドを実行します。<br>
Machine Learning Studio の ジョブに登録されていることを確認します。

In [ ]:
# submit the job to the backend
returned_job = ml_client.jobs.create_or_update(classification_job)

print(f"Created job: {returned_job}")

## 6. Retrieve the Best Trial (Best Model's trial/run)
MLFLowClient を使用して、以前に完了した AutoML トレーニングの結果（モデル、成果物、メトリクスなど）にアクセスします。<br>
この作業は Machine Learning Studio UI から行うこともできます。


### 6.1. Obtain the tracking URI for MLFlow

In [ ]:
import mlflow

# Obtain the tracking URL from MLClient
MLFLOW_TRACKING_URI = ml_client.workspaces.get(
    name=ml_client.workspace_name
).mlflow_tracking_uri

print(MLFLOW_TRACKING_URI)

In [ ]:
# Set the MLFLOW TRACKING URI

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

print("\nCurrent tracking uri: {}".format(mlflow.get_tracking_uri()))

In [ ]:
from mlflow.tracking.client import MlflowClient
from mlflow.artifacts import download_artifacts

# Initialize MLFlow client
mlflow_client = MlflowClient()

### 6.2. Get the AutoML parent Job

In [ ]:
job_name = returned_job.name

# Example if providing an specific Job name/ID
# job_name = "b4e95546-0aa1-448e-9ad6-002e3207b4fc"

# Get the parent run
mlflow_parent_run = mlflow_client.get_run(job_name)

print("Parent Run: ")
print(mlflow_parent_run)

In [ ]:
# Print parent run tags. 'automl_best_child_run_id' tag should be there.
print(mlflow_parent_run.data.tags)

### 6.3. Get the AutoML best child run
親の Run を取得できたら最も精度の高い子 Run を取得します。

In [ ]:
# Get the best model's child run

best_child_run_id = mlflow_parent_run.data.tags["automl_best_child_run_id"]
print("Found best child run id: ", best_child_run_id)

best_run = mlflow_client.get_run(best_child_run_id)

print("Best child run: ")
print(best_run)

### 6.4. Get best model run's metrics
過去に実行したAutoMLの結果（モデル、成果物、メトリクスなど）にアクセスします。

In [ ]:
best_run.data.metrics

### 6.5. Download the best model locally
Machine Learning Studio の UI から直接ダウンロードもできます。

In [ ]:
import os

# Create local folder
local_dir = "./artifact_downloads"
if not os.path.exists(local_dir):
    os.mkdir(local_dir)

In [ ]:
# Download run's artifacts/outputs
local_path = download_artifacts(
    run_id=best_run.info.run_id, artifact_path="outputs", dst_path=local_dir
)
print("Artifacts downloaded in: {}".format(local_path))
print("Artifacts: {}".format(os.listdir(local_path)))

In [ ]:
# Show the contents of the MLFlow model folder
os.listdir("./artifact_downloads/outputs")

## 7. Predict models
pkl 形式で保存したモデルと onnx 形式で保存したモデルで推論結果を比較します。

### 7.1. load model using joblib

In [ ]:
import joblib

model_path = './artifact_downloads/outputs/model.pkl'
model = joblib.load(model_path)

In [ ]:
model

In [ ]:
# Train a model.
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()
X, y = iris.data, iris.target 
X_train, X_test, y_train, y_test = train_test_split(X, y)


In [ ]:
import pandas as pd
from sklearn import datasets
#data_sample = PandasParameterType(pd.DataFrame({"SepalLengthCm": pd.Series([0.0], dtype="float64"), "SepalWidthCm": pd.Series([0.0], dtype="float64"), "PetalLengthCm": pd.Series([0.0], dtype="float64"), "PetalWidthCm": pd.Series([0.0], dtype="float64")}))

iris = datasets.load_iris()
feature_names = [
        'SepalLengthCm',
        'SepalWidthCm',
        'PetalLengthCm',
        'PetalWidthCm']
df = pd.DataFrame(iris.data, columns=feature_names)
df.head()



In [ ]:
model.predict(df[70:71])


### 7.2. load model using onnx
Open Neural Network Exchange (ONNX) を使用して、Azure Machine Learning の自動機械学習 (AutoML) から生成されたモデルで予測を行う方法について説明します。

予測に ONNX を使用するには、次のことを行う必要があります。

- AutoML トレーニングの実行から ONNX モデル ファイルをダウンロードします
- ONNX モデルの入力と出力を理解します
- 入力に必要な形式になるようにデータを前処理します
- Python 用の ONNX ランタイムで推論を実行します


ONNX Runtime は、クロスプラットフォームの推論をサポートするオープン ソース プロジェクトです。 ONNX ランタイムでは、複数のプログラミング言語 (Python、C++、C#、C、Java、JavaScript など) にわたって API が提供されます。 これらの API を使用して、入力データに対する推論を実行できます。 ONNX 形式でエクスポートされたモデルの作成後、プロジェクトに必要な任意のプログラミング言語でこれらの API を使用できます。

In [ ]:
#onnxruntime パッケージをインストールします。
#%pip install onnxruntime

ダウンロードした ONNX モデル ファイル `model.onnx` をロードします。
ロードしたら、ONNX モデルの入力と出力を理解します。

In [ ]:
# Compute the prediction with ONNX Runtime
import onnxruntime as rt
import numpy as np

sess = rt.InferenceSession("./artifact_downloads/outputs/model.onnx", providers=["CPUExecutionProvider"])
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
print(input_name, label_name)

In [ ]:
for ix in sess.get_inputs():
    print(ix.name)

ONNX モデルで想定される入力形式に合わせます。

In [ ]:
input_values = [5.9, 3.2, 4.8, 1.8]

input_data = {
    'SepalLengthCm': np.array([[input_values[0]]], dtype=np.float32),
    'SepalWidthCm':  np.array([[input_values[1]]], dtype=np.float32),
    'PetalLengthCm': np.array([[input_values[2]]], dtype=np.float32),
    'PetalWidthCm':  np.array([[input_values[3]]], dtype=np.float32)
}

推論を実行します

In [ ]:
pred_onx = sess.run([label_name], input_data)[0]
pred_onx